# PITiness Assessment Methods for IRB PD Models

This notebook implements various methods to quantify the Point-in-Time (PIT) characteristics of Probability of Default (PD) models.

**Methods Implemented:**
1. PRA Cyclicality Formula (Bank of England SS11/13)
2. Correlation Test (PD vs DR)
3. Variance Ratio
4. Regression Beta
5. Carlehed-Petrov λ (PITness parameter)
6. Z-Score Volatility Ratio
7. Migration-Based PITiness (IIF/GCD approach)

**Interpretation Guide:**
- Values closer to **1 (or 100%)** indicate **PIT** behavior
- Values closer to **0 (or 0%)** indicate **TTC** behavior

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.optimize import minimize_scalar
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.float_format', '{:.4f}'.format)
np.random.seed(42)

## 1. Sample Data Generation

We create realistic time series for multiple countries with varying degrees of PITiness.
- **Actual DR**: Observed default rates (ground truth)
- **Model PD**: Estimated PDs from the model being assessed

In [ ]:
def generate_pd_dr_series(n_years=15, base_dr=0.02, volatility=0.3, pitiness=0.7, seed=None):
    """
    Generate correlated PD and DR time series with controlled PITiness.
    
    Parameters:
    -----------
    n_years : int - Number of annual observations
    base_dr : float - Long-run average default rate
    volatility : float - Volatility of default rates
    pitiness : float - True PITiness (0=TTC, 1=PIT)
    
    Returns:
    --------
    tuple: (actual_dr, model_pd) arrays
    """
    if seed:
        np.random.seed(seed)
    
    # Generate economic cycle (systematic factor Z)
    z_factor = np.zeros(n_years)
    z_factor[0] = np.random.normal(0, 1)
    for t in range(1, n_years):
        z_factor[t] = 0.7 * z_factor[t-1] + np.random.normal(0, 0.7)  # AR(1) process
    
    # Generate actual default rates using Vasicek framework
    rho = 0.15  # Asset correlation
    k = stats.norm.ppf(base_dr)
    
    actual_dr = np.zeros(n_years)
    for t in range(n_years):
        # PIT default rate from Vasicek
        threshold = (k - np.sqrt(rho) * z_factor[t]) / np.sqrt(1 - rho)
        actual_dr[t] = stats.norm.cdf(threshold)
        # Add idiosyncratic noise
        actual_dr[t] *= (1 + np.random.normal(0, 0.1))
    
    actual_dr = np.clip(actual_dr, 0.001, 0.30)  # Bound DRs
    
    # Generate model PDs based on PITiness
    ttc_pd = base_dr * np.ones(n_years)  # Pure TTC = constant
    pit_pd = actual_dr.copy()  # Pure PIT = tracks DR
    
    # Hybrid: blend based on pitiness parameter
    model_pd = pitiness * pit_pd + (1 - pitiness) * ttc_pd
    model_pd += np.random.normal(0, 0.002, n_years)  # Add noise
    model_pd = np.clip(model_pd, 0.001, 0.30)
    
    return actual_dr, model_pd, z_factor

In [ ]:
# Generate data for multiple countries with different PITiness characteristics
countries_config = {
    'UK_Mortgages': {'base_dr': 0.015, 'volatility': 0.25, 'pitiness': 0.25, 'seed': 1},
    'Germany_Corporate': {'base_dr': 0.025, 'volatility': 0.35, 'pitiness': 0.60, 'seed': 2},
    'Spain_SME': {'base_dr': 0.045, 'volatility': 0.45, 'pitiness': 0.85, 'seed': 3},
    'France_Retail': {'base_dr': 0.030, 'volatility': 0.30, 'pitiness': 0.50, 'seed': 4},
    'Italy_Corporate': {'base_dr': 0.040, 'volatility': 0.40, 'pitiness': 0.70, 'seed': 5},
    'Netherlands_Mortgage': {'base_dr': 0.012, 'volatility': 0.20, 'pitiness': 0.20, 'seed': 6},
    'Poland_Consumer': {'base_dr': 0.055, 'volatility': 0.50, 'pitiness': 0.90, 'seed': 7},
    'Sweden_Corporate': {'base_dr': 0.020, 'volatility': 0.28, 'pitiness': 0.40, 'seed': 8},
}

# Store generated data
data = {}
years = list(range(2010, 2025))

for country, config in countries_config.items():
    dr, model_pd_arr, z = generate_pd_dr_series(
        n_years=len(years),
        base_dr=config['base_dr'],
        volatility=config['volatility'],
        pitiness=config['pitiness'],
        seed=config['seed']
    )
    data[country] = {
        'actual_dr': dr,
        'model_pd': model_pd_arr,
        'z_factor': z,
        'true_pitiness': config['pitiness']
    }

# Display sample data
sample_df = pd.DataFrame({
    'Year': years,
    'UK_DR': data['UK_Mortgages']['actual_dr'],
    'UK_PD': data['UK_Mortgages']['model_pd'],
    'Germany_DR': data['Germany_Corporate']['actual_dr'],
    'Germany_PD': data['Germany_Corporate']['model_pd'],
})
print("Sample Data (first 5 years):")
print(sample_df.head())

## 2. PITiness Calculation Methods

### Method 1: PRA Cyclicality Formula (Bank of England SS11/13)

$$\text{Cyclicality\%} = \frac{\sum|PD_t - PD_{t-1}|}{\sum|DR_t - DR_{t-1}|} \times 100$$

This measures what proportion of DR volatility is captured by PD movements.

In [ ]:
def pra_cyclicality(model_pd, actual_dr):
    """
    PRA Cyclicality Formula from SS11/13.
    
    Measures the proportion of default rate changes explained by PD changes.
    
    Returns: Cyclicality percentage (0-100%, higher = more PIT)
    """
    pd_changes = np.abs(np.diff(model_pd))
    dr_changes = np.abs(np.diff(actual_dr))
    
    # Avoid division by zero
    if np.sum(dr_changes) < 1e-10:
        return np.nan
    
    cyclicality = np.sum(pd_changes) / np.sum(dr_changes)
    return min(cyclicality, 1.0)  # Cap at 100%

### Method 2: Correlation Test

$$\text{PITiness} = Corr(PD_{model}, DR_{actual})$$

Simple Pearson correlation between model PDs and observed default rates.

In [ ]:
def correlation_pitiness(model_pd, actual_dr):
    """
    Correlation-based PITiness measure.
    
    Returns: Correlation coefficient (0-1, higher = more PIT)
    """
    corr, _ = stats.pearsonr(model_pd, actual_dr)
    return max(0, corr)  # Floor at 0 for interpretability

### Method 3: Variance Ratio

$$\text{PITiness} = \frac{Var(PD_{model})}{Var(DR_{actual})}$$

Ratio of model PD variance to actual DR variance.

In [ ]:
def variance_ratio_pitiness(model_pd, actual_dr):
    """
    Variance ratio PITiness measure.
    
    Returns: Variance ratio (0-1+, values near 1 indicate PIT)
    """
    var_pd = np.var(model_pd)
    var_dr = np.var(actual_dr)
    
    if var_dr < 1e-10:
        return np.nan
    
    ratio = var_pd / var_dr
    return min(ratio, 1.0)  # Cap at 1

### Method 4: Regression Beta

$$PD_{model} = \alpha + \beta \cdot DR_{actual} + \varepsilon$$

The slope coefficient β indicates how strongly PDs respond to DR changes.

In [ ]:
def regression_beta_pitiness(model_pd, actual_dr):
    """
    Regression-based PITiness (beta coefficient).
    
    Returns: Beta coefficient (0-1, higher = more PIT)
    """
    slope, intercept, r_value, p_value, std_err = stats.linregress(actual_dr, model_pd)
    return max(0, min(slope, 1.0))  # Bound between 0 and 1

### Method 5: Carlehed-Petrov λ (PITness Parameter)

Based on the Vasicek framework:

$$PD_{hybrid} = \Phi\left[\frac{\Phi^{-1}(PD_{TTC}) - \lambda\sqrt{\rho} \cdot Z}{\sqrt{1 - \lambda^2\rho}}\right]$$

We estimate λ by minimizing the squared error between model PDs and the theoretical hybrid PDs.

In [ ]:
def carlehed_petrov_lambda(model_pd, actual_dr, rho=0.15):
    """
    Carlehed-Petrov PITness parameter (λ) estimation.
    
    Uses optimization to find λ that best explains the model PD behavior.
    
    Parameters:
    -----------
    model_pd : array - Model PD estimates
    actual_dr : array - Observed default rates
    rho : float - Asset correlation parameter (default 0.15)
    
    Returns: λ (0-1, higher = more PIT)
    """
    # Estimate TTC PD as long-run average
    pd_ttc = np.mean(actual_dr)
    k = stats.norm.ppf(pd_ttc)
    
    # Extract Z factors from observed DRs
    z_factors = np.zeros(len(actual_dr))
    for t in range(len(actual_dr)):
        dr_t = np.clip(actual_dr[t], 0.0001, 0.9999)
        z_factors[t] = (k - np.sqrt(1 - rho) * stats.norm.ppf(dr_t)) / np.sqrt(rho)
    
    def objective(lambda_param):
        """Minimize squared error between model and theoretical hybrid PD"""
        hybrid_pd = np.zeros(len(actual_dr))
        for t in range(len(actual_dr)):
            denom = np.sqrt(1 - lambda_param**2 * rho)
            if denom < 0.01:
                denom = 0.01
            numer = k - lambda_param * np.sqrt(rho) * z_factors[t]
            hybrid_pd[t] = stats.norm.cdf(numer / denom)
        
        return np.sum((model_pd - hybrid_pd)**2)
    
    # Optimize λ
    result = minimize_scalar(objective, bounds=(0, 1), method='bounded')
    return result.x

### Method 6: Z-Score Volatility Ratio

$$\text{PITiness} = \frac{Std(\Phi^{-1}(PD_{model}))}{Std(\Phi^{-1}(DR_{actual}))}$$

Compares the volatility of transformed PDs to transformed DRs in the normal space.

In [ ]:
def zscore_volatility_ratio(model_pd, actual_dr):
    """
    Z-score volatility ratio PITiness measure.
    
    Compares volatility in normal (probit) space.
    
    Returns: Volatility ratio (0-1, higher = more PIT)
    """
    # Transform to normal space (probit)
    pd_clipped = np.clip(model_pd, 0.0001, 0.9999)
    dr_clipped = np.clip(actual_dr, 0.0001, 0.9999)
    
    z_pd = stats.norm.ppf(pd_clipped)
    z_dr = stats.norm.ppf(dr_clipped)
    
    std_z_pd = np.std(z_pd)
    std_z_dr = np.std(z_dr)
    
    if std_z_dr < 1e-10:
        return np.nan
    
    ratio = std_z_pd / std_z_dr
    return min(ratio, 1.0)

### Method 7: First-Difference Correlation

$$\text{PITiness} = Corr(\Delta PD_t, \Delta DR_t)$$

Correlation of year-over-year changes (captures co-movement dynamics).

In [ ]:
def first_diff_correlation(model_pd, actual_dr):
    """
    First-difference correlation PITiness.
    
    Measures co-movement of changes rather than levels.
    
    Returns: Correlation of changes (0-1, higher = more PIT)
    """
    pd_diff = np.diff(model_pd)
    dr_diff = np.diff(actual_dr)
    
    if len(pd_diff) < 3:
        return np.nan
    
    corr, _ = stats.pearsonr(pd_diff, dr_diff)
    return max(0, corr)

### Method 8: R-Squared from Regression

$$R^2 = 1 - \frac{SS_{res}}{SS_{tot}}$$

Proportion of DR variance explained by the model PD.

In [ ]:
def r_squared_pitiness(model_pd, actual_dr):
    """
    R-squared from regressing DR on model PD.
    
    Returns: R² (0-1, higher = more PIT)
    """
    slope, intercept, r_value, p_value, std_err = stats.linregress(model_pd, actual_dr)
    return r_value**2

## 3. Calculate PITiness for All Countries

In [ ]:
# Define all methods
methods = {
    'PRA_Cyclicality': pra_cyclicality,
    'Correlation': correlation_pitiness,
    'Variance_Ratio': variance_ratio_pitiness,
    'Regression_Beta': regression_beta_pitiness,
    'Carlehed_Petrov_λ': carlehed_petrov_lambda,
    'ZScore_Vol_Ratio': zscore_volatility_ratio,
    'FirstDiff_Corr': first_diff_correlation,
    'R_Squared': r_squared_pitiness,
}

# Calculate PITiness for each country and method
results = []

for country, series_data in data.items():
    row = {'Country_Portfolio': country, 'True_PITiness': series_data['true_pitiness']}
    
    for method_name, method_func in methods.items():
        try:
            pitiness_value = method_func(
                series_data['model_pd'],
                series_data['actual_dr']
            )
            row[method_name] = pitiness_value
        except Exception as e:
            row[method_name] = np.nan
    
    results.append(row)

# Create results DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.set_index('Country_Portfolio')

In [ ]:
# Display main results table
print("="*100)
print("PITiness Assessment Results by Country/Portfolio and Method")
print("="*100)
print("\nInterpretation: Values closer to 1.0 indicate PIT behavior, closer to 0.0 indicate TTC behavior")
print("\n")
display_df = results_df.copy()
display_df

In [ ]:
# Create a styled version with conditional formatting
def color_pitiness(val):
    """Color code PITiness values: green=PIT, red=TTC"""
    if pd.isna(val):
        return ''
    if val > 0.7:
        return 'background-color: #90EE90'  # Light green (PIT)
    elif val > 0.4:
        return 'background-color: #FFFFE0'  # Light yellow (Hybrid)
    else:
        return 'background-color: #FFB6C1'  # Light red (TTC)

styled_df = results_df.style.applymap(
    color_pitiness, 
    subset=[col for col in results_df.columns if col != 'True_PITiness']
)
styled_df

## 4. Method Comparison and Validation

In [ ]:
# Calculate correlation of each method with true PITiness
method_cols = [col for col in results_df.columns if col != 'True_PITiness']

method_performance = {}
for method in method_cols:
    valid_mask = ~results_df[method].isna()
    if valid_mask.sum() > 2:
        corr, _ = stats.pearsonr(
            results_df.loc[valid_mask, 'True_PITiness'],
            results_df.loc[valid_mask, method]
        )
        mae = np.mean(np.abs(
            results_df.loc[valid_mask, 'True_PITiness'] - 
            results_df.loc[valid_mask, method]
        ))
        method_performance[method] = {'Correlation_with_True': corr, 'MAE': mae}

performance_df = pd.DataFrame(method_performance).T
performance_df = performance_df.sort_values('Correlation_with_True', ascending=False)

print("\n" + "="*60)
print("Method Performance vs True PITiness")
print("="*60)
print("\nHigher correlation and lower MAE indicate better method accuracy\n")
performance_df

In [ ]:
# Summary statistics by method
print("\n" + "="*60)
print("Summary Statistics by Method")
print("="*60 + "\n")

summary_stats = results_df[method_cols].describe().T
summary_stats['range'] = summary_stats['max'] - summary_stats['min']
summary_stats

## 5. Detailed Output Table

In [ ]:
# Create final comprehensive table
final_table = results_df.copy()

# Add average across methods (excluding True_PITiness)
final_table['Avg_Estimated'] = final_table[method_cols].mean(axis=1)
final_table['Std_Across_Methods'] = final_table[method_cols].std(axis=1)

# Reorder columns
col_order = ['True_PITiness'] + method_cols + ['Avg_Estimated', 'Std_Across_Methods']
final_table = final_table[col_order]

print("\n" + "="*120)
print("FINAL PITiness ASSESSMENT TABLE")
print("="*120)
print("\nRows: Country/Portfolio pairs | Columns: PITiness methods")
print("Values: PITiness score (0=TTC, 1=PIT)")
print("\n")
final_table.round(4)

In [ ]:
# Export to CSV for further analysis
final_table.to_csv('pitiness_results.csv')
print("\nResults exported to 'pitiness_results.csv'")

## 6. Method Documentation Summary

| Method | Formula | Pros | Cons |
|--------|---------|------|------|
| **PRA Cyclicality** | Σ\|ΔPD\|/Σ\|ΔDR\| | Regulatory aligned (UK); intuitive | Volatile with short series |
| **Correlation** | Corr(PD, DR) | Simple; robust | Doesn't capture magnitude |
| **Variance Ratio** | Var(PD)/Var(DR) | Captures volatility matching | Sensitive to outliers |
| **Regression Beta** | β from PD=α+βDR | Captures responsiveness | Assumes linearity |
| **Carlehed-Petrov λ** | Vasicek framework | Theoretically rigorous | Requires ρ assumption |
| **Z-Score Vol Ratio** | Std(Φ⁻¹PD)/Std(Φ⁻¹DR) | Works in normal space | Transformation sensitive |
| **First-Diff Corr** | Corr(ΔPD, ΔDR) | Captures dynamics | Loses level information |
| **R-Squared** | R² from regression | Standard statistical measure | Doesn't indicate direction |

## 7. Visualization

In [ ]:
import matplotlib.pyplot as plt

# Calculate summary columns first
results_df['Avg_Estimated'] = results_df[method_cols].mean(axis=1)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Heatmap of results
ax1 = axes[0, 0]
heatmap_data = results_df[method_cols].values
im = ax1.imshow(heatmap_data, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
ax1.set_xticks(range(len(method_cols)))
ax1.set_xticklabels([m.replace('_', '\n') for m in method_cols], rotation=45, ha='right', fontsize=8)
ax1.set_yticks(range(len(results_df)))
ax1.set_yticklabels(results_df.index, fontsize=8)
ax1.set_title('PITiness Heatmap by Method and Portfolio')
plt.colorbar(im, ax=ax1, label='PITiness (0=TTC, 1=PIT)')

# Plot 2: True vs Estimated PITiness
ax2 = axes[0, 1]
ax2.scatter(results_df['True_PITiness'], results_df['Avg_Estimated'], s=100, alpha=0.7)
ax2.plot([0, 1], [0, 1], 'r--', label='Perfect estimation')
for idx, row in results_df.iterrows():
    ax2.annotate(idx.split('_')[0], (row['True_PITiness'], row['Avg_Estimated']), fontsize=7)
ax2.set_xlabel('True PITiness')
ax2.set_ylabel('Average Estimated PITiness')
ax2.set_title('True vs Estimated PITiness')
ax2.legend()
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)

# Plot 3: Method comparison boxplot
ax3 = axes[1, 0]
results_df[method_cols].boxplot(ax=ax3, rot=45)
ax3.set_ylabel('PITiness Score')
ax3.set_title('Distribution of PITiness Scores by Method')
ax3.set_xticklabels([m.replace('_', '\n') for m in method_cols], fontsize=8)

# Plot 4: Sample time series
ax4 = axes[1, 1]
sample_country = 'Germany_Corporate'
ax4.plot(years, data[sample_country]['actual_dr']*100, 'b-o', label='Actual DR', linewidth=2)
ax4.plot(years, data[sample_country]['model_pd']*100, 'r--s', label='Model PD', linewidth=2)
ax4.set_xlabel('Year')
ax4.set_ylabel('Rate (%)')
ax4.set_title(f'Sample: {sample_country}\n(True PITiness: {data[sample_country]["true_pitiness"]:.0%})')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('pitiness_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nVisualization saved to 'pitiness_analysis.png'")

## 8. Applying to Your Own Data

To use these methods with your own data, replace the synthetic data with your actual time series:

In [ ]:
# Example: How to use with your own data

def calculate_all_pitiness_metrics(model_pd_series, actual_dr_series, rho=0.15):
    """
    Calculate all PITiness metrics for a single portfolio.
    
    Parameters:
    -----------
    model_pd_series : array-like - Time series of model PD estimates
    actual_dr_series : array-like - Time series of observed default rates
    rho : float - Asset correlation for Carlehed-Petrov method
    
    Returns:
    --------
    dict : Dictionary of PITiness metrics
    """
    model_pd = np.array(model_pd_series)
    actual_dr = np.array(actual_dr_series)
    
    results = {
        'PRA_Cyclicality': pra_cyclicality(model_pd, actual_dr),
        'Correlation': correlation_pitiness(model_pd, actual_dr),
        'Variance_Ratio': variance_ratio_pitiness(model_pd, actual_dr),
        'Regression_Beta': regression_beta_pitiness(model_pd, actual_dr),
        'Carlehed_Petrov_λ': carlehed_petrov_lambda(model_pd, actual_dr, rho),
        'ZScore_Vol_Ratio': zscore_volatility_ratio(model_pd, actual_dr),
        'FirstDiff_Corr': first_diff_correlation(model_pd, actual_dr),
        'R_Squared': r_squared_pitiness(model_pd, actual_dr),
    }
    
    return results

# Example usage:
# your_results = calculate_all_pitiness_metrics(
#     model_pd_series=your_pd_data,
#     actual_dr_series=your_dr_data,
#     rho=0.15
# )

print("Function 'calculate_all_pitiness_metrics' ready for use with your data.")
print("\nExample output for UK_Mortgages:")
example_results = calculate_all_pitiness_metrics(
    data['UK_Mortgages']['model_pd'],
    data['UK_Mortgages']['actual_dr']
)
for metric, value in example_results.items():
    print(f"  {metric}: {value:.4f}")